# Vector to Sequence RNN (Part-ii)

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
class CONFIG(object):
  """CONFIG"""
  def __init__(self):
    self.img_size = (256,256)
    self.base= '../input/bms-molecular-translation/'
    self.df= '../input/bms-molecular-translation/train_labels.csv'
    self.train='../input/bmsdataversion2/BMS-Datav2/train/'
    self.batch_size= 16
    self.lr= 0.001
    self.val_split= 0.1
    self.seed= 22
    self.n_epochs= 4
    self.vocab_size= 600
    
    
cfg= CONFIG()

def load_path(img_id):
    return img_id[0] +'/'+img_id[1]+'/'+img_id[2] +'/'+img_id+'.png'

In [ ]:
vocab= ['<start>', '<end>', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
        '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
        '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
        '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58',
        '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73',
        '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
        '89', '90', '91', '92', '(', ')', '-', 'c']
len(vocab)

## INPUT DATA
<img src="https://camo.githubusercontent.com/c73259c22376b40060d05571a8731781d1058771/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6a656666686561746f6e2f7438315f3535385f646565705f6c6561726e696e672f6d61737465722f696d616765732f63617074696f6e2d322e706e67" height="300" align="left">

In [ ]:
img_paths= np.load('../input/bmsdataversion2/BMS-Datav2/path.npy')
X= np.load('../input/bmsdataversion2/BMS-Datav2/X.npy')
Y= np.load('../input/bmsdataversion2/BMS-Datav2/Y.npy')

X.shape, Y.shape, img_paths.shape

In [ ]:
Y= Y.astype(np.float32)
X= X.astype(np.float32)

In [ ]:
img_paths= (pd.Series(img_paths).apply(lambda x: x.split('/')[-1]))
img_paths[0]

In [ ]:
plt.bar(list(range(98)),Y.sum(0))
plt.title('Class Distribution')
plt.show()

In [ ]:
plt.bar(list(range(94)),Y.sum(0)[0:-4])
plt.title('Class Distribution')
plt.show()

## Define Model

In [ ]:
def build_imgext():
    img_base= tf.keras.applications.ResNet50V2(include_top=False)
    inp= layers.Input((cfg.img_size[0], cfg.img_size[1], 3))
    x= img_base(inp)
    x= layers.Dropout(0.3)(x)
    x= layers.GlobalAveragePooling2D()(x)
    x= layers.Dense(512, 'relu')(x)
    return tf.keras.Model(inp, x)

def build_model():
    img_base= build_imgext()
    inp1= layers.Input((cfg.img_size[0], cfg.img_size[1], 3))
    fc1= img_base(inp1)
    
    inp2 = layers.Input(shape=(len(vocab)))
    se1 = layers.Embedding(len(vocab), cfg.vocab_size, mask_zero=True)(inp2)
    se2 = layers.Dropout(0.25)(se1)
    se3 = layers.LSTM(512)(se2)
    
    decoder1 = layers.add([fc1, se3])
    decoder2 = layers.Dense(512, activation='relu')(decoder1)
    out = layers.Dense(len(vocab), activation='softmax')(decoder2)
    
    return tf.keras.Model([inp1, inp2], out)

## TPU PIPELINE

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
img_size= cfg.img_size[0]
def build_decoder(with_labels=True, target_size=(img_size, img_size), ext='png'):
    def decode(path):
        file_bytes = tf.io.read_file(path) # Reads and outputs the entire contents of the input filename.

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3) # Decode a PNG-encoded image to a uint8 or uint16 tensor
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3) # Decode a JPEG-encoded image to a uint8 tensor
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # Casts a tensor to the type float32 and divides by 255.
        img = tf.image.resize(img, target_size) # Resizing to target size
        return img
    
    def decode_with_labels(path, x, label):
        return (decode(path), x), label
    
    return decode_with_labels if with_labels else decode

In [ ]:
def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, x=None, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, x, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    #dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize, drop_remainder=False).prefetch(AUTO)
    # dset = dset.batch(bsize, drop_remainder=False).prefetch(AUTO) #overlaps data preprocessing and model execution while training
    return dset

In [ ]:
DATASET_NAME = "bmsdataversion2"
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * cfg.batch_size
print('batch size', batch_size)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path(DATASET_NAME)
GCS_DS_PATH

In [ ]:
Ydf= pd.DataFrame(Y)
# end tags
ind= Ydf[Ydf[1]==1].index
sp= ind[12000] +1

In [ ]:
img_paths= GCS_DS_PATH + '/BMS-Datav2/train/' + img_paths
#img_paths= '../input/bmsdataversion2/BMS-Datav2/train/' + img_paths
img_paths[0]

In [ ]:
#Split data
(train_paths, valid_paths, 
 train_labels, valid_labels,
 X_train, X_valid) = (img_paths[:sp], img_paths[sp:], Y[:sp], Y[sp:], X[:sp], X[sp:])

print(train_paths.shape, valid_paths.shape)

In [ ]:
decoder = build_decoder(with_labels=True, target_size=(img_size, img_size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, X_train, train_labels, bsize=batch_size, decode_fn=decoder)

dvalid = build_dataset(
    valid_paths, X_valid, valid_labels, bsize=batch_size, 
    repeat=False, shuffle=False, augment=False, decode_fn=decoder)

In [ ]:
data, _ = dtrain.take(2)
images = data[0][0].numpy()

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(15,10))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img, aspect= True)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
with strategy.scope():
    model= build_model()
    loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.0)
    
    model.compile(tf.keras.optimizers.Adam(lr=cfg.lr),
                  loss= loss, metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
name= 'img_capv1.h5'

rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)
        
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

steps_per_epoch = (train_paths.shape[0] // batch_size)//2
steps_per_epoch

In [ ]:
history = model.fit(dtrain,                      
                    validation_data=dvalid,                                       
                    epochs=cfg.n_epochs,
                    callbacks=[rlr,es,ckp],
                    steps_per_epoch=steps_per_epoch,
                    verbose=1)

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( history.history["loss"], label = "Training Loss", marker='o')
plt.plot( history.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("ACC")
plt.plot( history.history["accuracy"], label = "Training ACC" , marker='o')
plt.plot( history.history["val_accuracy"], label = "Validation ACC", marker='+')
plt.grid(True)
plt.legend()
plt.show()

![](https://i.gifer.com/7ImI.gif)